In [3]:
import regex as reg
import numpy as np
import pandas as pd
import requests as re
from bs4 import BeautifulSoup as bs


c:\Users\hirsh\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\hirsh\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\hirsh\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
# website=re.get("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats")
# soup=bs(website)
df=pd.read_html("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats")[0]

In [9]:
def get_team_link_prev(name):
    # website=re.get("https://fbref.com/en/comps/9/Premier-League-Stats").text
    website=re.get("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats").text

    soup = bs(website)
    team_links=dict()
    for body in soup.find_all("tbody"):
        for row in body.find_all("tr"):
            for a_tag in row.find('td').find_all('a'):
                if a_tag is not None:
                    team_link="https://fbref.com/"+a_tag["href"]
                    team_name=reg.search(r'/([^/]+)-Stats$',team_link)
                    if team_name:
                        team_name = team_name.group(1)
                        # print(result)
                        temp=team_link.split('/')[:-1]
                        temp.append('2022-2023')
                        temp.append(team_name)
                        team_link='/'.join(temp)
                    team_links[team_name]=team_link
    if name in team_links.keys():
        return team_links[name]
    print("Incorrect name ,Recheck the team Name, ")

def get_team_statistics(name,table_name=""):
    team_link=get_team_link_prev(name)
    #Standard Stats
    # individual_team = bs(re.get(get_team_link_prev(name)).text)
    if table_name=='Standard Stats':
        individual_team_standard_stats = pd.read_html(team_link,match="Standard Stats")[0]
        individual_team_standard_stats.columns = [col[1] for col in individual_team_standard_stats.columns]
        individual_team_standard_stats = individual_team_standard_stats.drop(columns="Matches")
        individual_team_standard_stats = individual_team_standard_stats.iloc[:-2,:]
        display(individual_team_standard_stats.head(5))
        return(individual_team_standard_stats)

    #Goalkeeping Statistics
    if table_name=='Goalkeeping Stats':
        goalkeeping_table = pd.read_html(team_link,match="Advanced Goalkeeping ")[0]
        goalkeeping = goalkeeping_table.iloc[:-2,:]
        goalkeeping.columns=goalkeeping.columns.droplevel(0)
        display(goalkeeping.head(5))
        return goalkeeping

    #Shooting stats
    if table_name=="shooting":
        Shooting_table = pd.read_html(team_link,match="Shooting ")[0]
        Shooting = Shooting_table.iloc[:-2,:]
        Shooting.columns=Shooting.columns.droplevel(0)
        display(Shooting.head(5))
        return Shooting
    
    if table_name=="Scores & Fixtures":
        scores_fixtures= pd.read_html(team_link,match="Scores & Fixtures")[0]
        # scores_fixtures.columns = scores_fixtures.columns.droplevel(0)
        display(scores_fixtures.head(5))
        return scores_fixtures
    
    #Passibg stats
    if table_name=="passing":      
        website=re.get(team_link).text

        soup = bs(website)
        poss =soup.find_all(class_="")
        passing_stats_bool = False
        for log in soup.find_all('a'):
            try:
                if 'passing' in log["href"] and "Premier-League" in log['href']:
                    new_link="https://fbref.com/"+log['href']
                    passing_stats=pd.read_html(new_link,match="Passing")[0]
                    passing_stats.columns=passing_stats.columns.droplevel()
                    passing_stats=passing_stats.iloc[:,:14]                   
                    passing_stats['Cmp%']=passing_stats["Cmp%"].astype('float')
                    passing_stats_bool =True
                    break
            except:
                passing_stats_bool=False
        if passing_stats_bool==True:
            return passing_stats
    

In [11]:
# print(get_team_link_prev('Manchester-United'))
display(get_team_statistics('Liverpool','shooting'))

,Player,Nation,Pos,Age,90s,Gls,Sh,SoT,SoT%,Sh/90,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,Alisson,br BRA,GK,29.0,37.0,0,0,0,NaN,0.00,...,NaN,0,0,0,0.0,0.0,NaN,0.0,0.0,Matches
1,Mohamed Salah,eg EGY,FW,30.0,36.6,19,121,45,37.2,3.31,...,15.1,2,2,4,21.7,18.5,0.15,-2.7,-1.5,Matches
2,Trent Alexander-Arnold,eng ENG,DF,23.0,32.5,2,40,8,20.0,1.23,...,23.8,5,0,0,2.4,2.4,0.06,-0.4,-0.4,Matches
3,Virgil van Dijk,nl NED,DF,31.0,31.5,3,30,9,30.0,0.95,...,12.8,0,0,0,2.6,2.6,0.09,0.4,0.4,Matches
4,Fabinho,br BRA,MF,28.0,29.7,0,18,4,22.2,0.61,...,22.0,0,0,0,0.7,0.7,0.04,-0.7,-0.7,Matches


,Player,Nation,Pos,Age,90s,Gls,Sh,SoT,SoT%,Sh/90,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,Alisson,br BRA,GK,29.0,37.0,0,0,0,NaN,0.00,...,NaN,0,0,0,0.0,0.0,NaN,0.0,0.0,Matches
1,Mohamed Salah,eg EGY,FW,30.0,36.6,19,121,45,37.2,3.31,...,15.1,2,2,4,21.7,18.5,0.15,-2.7,-1.5,Matches
2,Trent Alexander-Arnold,eng ENG,DF,23.0,32.5,2,40,8,20.0,1.23,...,23.8,5,0,0,2.4,2.4,0.06,-0.4,-0.4,Matches
3,Virgil van Dijk,nl NED,DF,31.0,31.5,3,30,9,30.0,0.95,...,12.8,0,0,0,2.6,2.6,0.09,0.4,0.4,Matches
4,Fabinho,br BRA,MF,28.0,29.7,0,18,4,22.2,0.61,...,22.0,0,0,0,0.7,0.7,0.04,-0.7,-0.7,Matches
5,Andrew Robertson,sct SCO,DF,28.0,28.7,0,13,2,15.4,0.45,...,14.5,0,0,0,1.3,1.3,0.11,-1.3,-1.3,Matches
6,Jordan Henderson,eng ENG,MF,32.0,23.0,0,23,5,21.7,1.00,...,22.8,0,0,0,1.4,1.4,0.06,-1.4,-1.4,Matches
7,Darwin Núñez,uy URU,FW,23.0,18.8,9,84,35,41.7,4.46,...,15.1,0,0,0,12.1,12.1,0.14,-3.1,-3.1,Matches
8,Harvey Elliott,eng ENG,"MF,FW",19.0,17.9,1,36,12,33.3,2.01,...,19.3,0,0,0,2.1,2.1,0.06,-1.1,-1.1,Matches
9,Ibrahima Konaté,fr FRA,DF,23.0,17.2,0,11,3,27.3,0.64,...,9.7,0,0,0,1.2,1.2,0.11,-1.2,-1.2,Matches


In [5]:
website=re.get("https://fbref.com/en/comps/9/Premier-League-Stats").text
soup = bs(website)
table =soup.find("table", class_="stats_table")
table

<table class="stats_table sortable min_width force_mobilize" data-cols-to-freeze=",2" id="results2023-202491_overall"> <caption>Regular season Table</caption> <colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup> <thead> <tr> <th aria-label="Rank" class="poptip sort_default_asc center" data-stat="rank" data-tip="&lt;strong&gt;Rank&lt;/strong&gt;&lt;br&gt;Squad finish in competition&lt;br&gt;Finish within the league or competition.&lt;br&gt;For knockout competitions may show final round reached.&lt;br&gt;Colors and arrows represent promotion/relegation or qualifiation for continental cups.&lt;br&gt;Trophy indicates team won league whether by playoffs or by leading the table.&lt;br&gt;Star indicates topped table in league USING another means of naming champion." scope="col">Rk</th> <th aria-label="Squad" class="poptip sort_default_asc center" data-stat="team" scope="col">Squad</th> <th aria-label="Mat

In [6]:
length = int(len(table.find('tr').text.split(" "))/2)
print(table.find('tr').text.strip(" ").split(" ")[0:length-1])



['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP']


In [7]:
team_links=dict()
for body in soup.find_all("tbody"):
    for row in body.find_all("tr"):
        for a_tag in row.find('td').find_all('a'):
            if a_tag is not None:
                team_link="https://fbref.com/"+a_tag["href"]
                team_name=reg.search(r'/([^/]+)-Stats$',team_link)
                if team_name:
                    team_name = team_name.group(1)
                    # print(result)
                team_links[team_name]=team_link
print(len(team_links))


20


In [8]:
df = pd.read_html("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats")[0]
regular_season_data=df.drop(columns="Notes")

link_df = pd.Series(team_links.values(),index=list(range(20)))
link_df

def convert(x):
    d={"W":"1","D":"0","L":'-1'}
    new_s=""
    for i in x:
        if i in d.keys():
            new_s+=d[i]
    return new_s
# 
regular_season_data["Last 5"]=regular_season_data["Last 5"].apply(lambda x: convert(x))
regular_season_data['Link']=link_df
regular_season_data
# regular_season_data["team_stat_link"] = team_links

KeyError: 'Last 5'

## Score & Fixtures analysis

In [ ]:
score_fix = get_team_statistics('Manchester-United','Scores & Fixtures')

score_fix[['GF','GA']] = score_fix[['GF','GA']].apply(lambda x: x[0])
score_fix.loc[:,['GF', 'GA', 'xG', 'xGA', 'Poss', 'Attendance']]=score_fix.astype({'GF':int,'GA':int,'xG':'Float32','xGA':'Float32','Poss':int,'Attendance':int})

stadium_scores=score_fix.groupby('Venue',as_index=False).agg({'GF':'sum','GA':'sum','Poss':'mean','Attendance':'mean','Date':'count'})
display(stadium_scores)

captain_group = score_fix.groupby('Captain',as_index=False).agg({'GF':'sum','GA':'sum','Poss':'mean','Attendance':'mean','Date':'count'})
display(captain_group)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2022-08-07,14:00,Premier League,Matchweek 1,Sun,Home,L,1,2,Brighton,1.4,1.5,62,73711,Harry Maguire,4-3-3,Paul Tierney,Match Report,NaN
1,2022-08-13,17:30,Premier League,Matchweek 2,Sat,Away,L,0,4,Brentford,0.9,1.6,66,17051,Harry Maguire,4-2-3-1,Stuart Attwell,Match Report,NaN
2,2022-08-22,20:00,Premier League,Matchweek 3,Mon,Home,W,2,1,Liverpool,1.8,1.3,30,74147,Bruno Fernandes,4-2-3-1,Michael Oliver,Match Report,NaN
3,2022-08-27,12:30,Premier League,Matchweek 4,Sat,Away,W,1,0,Southampton,1.1,1.4,51,31196,Bruno Fernandes,4-2-3-1,Andy Madley,Match Report,NaN
4,2022-09-01,20:00,Premier League,Matchweek 5,Thu,Away,W,1,0,Leicester City,1.5,0.7,47,32226,Bruno Fernandes,4-2-3-1,Craig Pawson,Match Report,NaN


C:\Users\hirsh\AppData\Local\Temp\ipykernel_29976\3916846521.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  score_fix.loc[:,['GF', 'GA', 'xG', 'xGA', 'Poss', 'Attendance']]=score_fix.astype({'GF':int,'GA':int,'xG':'Float32','xGA':'Float32','Poss':int,'Attendance':int})


,Venue,GF,GA,Poss,Attendance,Date
0,Away,26,52,55.961538,38980.692308,26
1,Home,34,68,56.205882,73569.558824,34
2,Neutral,2,4,39.500000,82312.000000,2


,Captain,GF,GA,Poss,Attendance,Date
0,Bruno Fernandes,44,88,53.772727,58154.181818,44
1,Cristiano Ronaldo,1,2,58.000000,42058.000000,1
2,Harry Maguire,16,32,61.312500,61776.250000,16
3,Wout Weghorst,1,2,40.000000,90225.000000,1


## Passing Statistics

In [ ]:
get_team_statistics("Manchester-United","passing").columns
# groupby('Result',as_index=False).agg({'GF':'mean'})


Index(['Date', 'Time', 'Round', 'Day', 'Venue', 'Result', 'GF', 'GA',
       'Opponent', 'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist'],
      dtype='object')

In [ ]:
result_vs_pass_completion=get_team_statistics("Newcastle-United","passing").groupby('Result',as_index=False).agg({'Cmp%':'mean'}).iloc[1:,:]
#No use
venue_vs_pass_completion=get_team_statistics("Manchester-City","passing").groupby('Venue',as_index=False).agg({'Cmp%':'mean'})
venue_vs_pass_completion

,Venue,Cmp%
0,Away,86.315789
1,Home,87.463158


In [ ]:
bs.get()